In [12]:
import pandas as pd
import re
from sklearn.ensemble import IsolationForest  # Para detección de anomalías
import dash
from dash import dcc, html
import plotly.express as px  # Para visualización
from datetime import datetime, timedelta

In [13]:
dir_path = "../process/"

# Entrenamiento

In [14]:

file=  "external_test_data" #"train_data" 

logs_df = pd.read_csv(f"{dir_path}3_22{file}_heuristic_prediction.csv", encoding='utf-8')

print(set(logs_df["Level"]))
print(logs_df.shape)
logs_df.head()

{'Advertencia', 'Error', 'Informativo'}
(320, 15)


,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly,Prioridad,PrioridadEtiqueta
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:47:44.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
1,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:52:15.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
2,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:52:52.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
3,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:53:16.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
4,jk2_init() Found child 6755 in scoreboard slot 6,Informativo,2005-12-04 04:53:29.000,3,Apache,child scoreboard slot,Depuración,Conectividad,Monitoreo,Único,48,4.0,1,7,Informativo


In [15]:
def obtener_estadisticas(df, columna):
    conteo = df[columna].value_counts(dropna=False)
    porcentaje = round(df[columna].value_counts(normalize=True, dropna=False) * 100, 2)
    resumen = pd.DataFrame({'Frecuencia': conteo, 'Porcentaje (%)': porcentaje})
    resumen.index.name = columna
    return resumen

# Aplicar a las columnas deseadas
estadisticas_severidad = obtener_estadisticas(logs_df, 'Severidad')
estadisticas_origen = obtener_estadisticas(logs_df, 'Origen')
estadisticas_tipo = obtener_estadisticas(logs_df, 'TipoEvento')
estadisticas_frecuencia = obtener_estadisticas(logs_df, 'Frecuencia')
estadisticas_anomalia = obtener_estadisticas(logs_df, 'Anomaly')


In [16]:
# Mostrar ejemplo
print("Severidad:")
print(estadisticas_severidad)

Severidad:
              Frecuencia  Porcentaje (%)
Severidad                               
Depuración           104           32.50
Crítico              100           31.25
Advertencia           80           25.00
No relevante          35           10.94
Información            1            0.31


In [17]:
# Mostrar ejemplo
print("Origen:")
print(estadisticas_origen)


Origen:
                     Frecuencia  Porcentaje (%)
Origen                                         
Aplicación                  105           32.81
Conectividad                103           32.19
Sistema operativo            57           17.81
Otro                         28            8.75
Gestión de energía           19            5.94
Sistema de archivos           8            2.50


In [18]:
# Mostrar ejemplo
print("Tipo:")
print(estadisticas_tipo)

Tipo:
                    Frecuencia  Porcentaje (%)
TipoEvento                                    
Errores                    135           42.19
Monitoreo                  103           32.19
Eventos                     41           12.81
Otro                        26            8.12
Transicion                  13            4.06
Parada de procesos           2            0.62


In [19]:
# Mostrar ejemplo
print("Frecuencia:")
print(estadisticas_frecuencia)

Frecuencia:
                     Frecuencia  Porcentaje (%)
Frecuencia                                     
Único                       139           43.44
Recurrente esperado         115           35.94
Recurrente anómalo           66           20.62


In [20]:
# Mostrar ejemplo
print("Anomaly:")
print(estadisticas_anomalia)

Anomaly:
         Frecuencia  Porcentaje (%)
Anomaly                            
 1              304            95.0
-1               16             5.0


# EV 

In [21]:

file="external_test_data"# "train_data" # 

logs_df = pd.read_csv(f"{dir_path}3_22{file}_heuristic_prediction.csv", encoding='utf-8')

print(set(logs_df["Level"]))
print(logs_df.shape)
logs_df.head()

{'Advertencia', 'Error', 'Informativo'}
(320, 15)


,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly,Prioridad,PrioridadEtiqueta
0,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:47:44.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
1,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:52:15.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
2,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:52:52.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
3,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:53:16.000,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1,11,Error
4,jk2_init() Found child 6755 in scoreboard slot 6,Informativo,2005-12-04 04:53:29.000,3,Apache,child scoreboard slot,Depuración,Conectividad,Monitoreo,Único,48,4.0,1,7,Informativo


In [22]:

# Aplicar a las columnas deseadas
estadisticas_severidad = obtener_estadisticas(logs_df, 'Severidad')
estadisticas_origen = obtener_estadisticas(logs_df, 'Origen')
estadisticas_tipo = obtener_estadisticas(logs_df, 'TipoEvento')
estadisticas_frecuencia = obtener_estadisticas(logs_df, 'Frecuencia')
estadisticas_anomalia = obtener_estadisticas(logs_df, 'Anomaly')

In [23]:
print("Severidad:")
print(estadisticas_severidad)

Severidad:
              Frecuencia  Porcentaje (%)
Severidad                               
Depuración           104           32.50
Crítico              100           31.25
Advertencia           80           25.00
No relevante          35           10.94
Información            1            0.31


In [24]:
print("Origen:")
print(estadisticas_origen)


Origen:
                     Frecuencia  Porcentaje (%)
Origen                                         
Aplicación                  105           32.81
Conectividad                103           32.19
Sistema operativo            57           17.81
Otro                         28            8.75
Gestión de energía           19            5.94
Sistema de archivos           8            2.50


In [25]:
print("Tipo:")
print(estadisticas_tipo)

Tipo:
                    Frecuencia  Porcentaje (%)
TipoEvento                                    
Errores                    135           42.19
Monitoreo                  103           32.19
Eventos                     41           12.81
Otro                        26            8.12
Transicion                  13            4.06
Parada de procesos           2            0.62


In [26]:
print("Frecuencia:")
print(estadisticas_frecuencia)

Frecuencia:
                     Frecuencia  Porcentaje (%)
Frecuencia                                     
Único                       139           43.44
Recurrente esperado         115           35.94
Recurrente anómalo           66           20.62


In [27]:
print("Anomaly:")
print(estadisticas_anomalia)

Anomaly:
         Frecuencia  Porcentaje (%)
Anomaly                            
 1              304            95.0
-1               16             5.0
